In [1]:
ENV["GKS_ENCODING"]="UTF-8"

using LinearAlgebra
using Statistics
using Plots
using JSON
using HDF5
using StatsPlots

gr(fontfamily="Helvetica")
default(fontfamily="Helvetica")

In [2]:
colors = [:orange, :black, :gold, :blue, :red, :brown, :bisque4, :gray, :blue, :black] #惑星の色
AU = 1.495978e8 #AU
G = 6.67430e-20 #gravitational constant (G)

6.6743e-20

In [76]:
function calc_orbit_ec(N, file, tEnd, dt, chunk_size, years)
    Nt = Int(ceil(tEnd/dt))
    ec = zeros(years)
    ec_i = 1
    chunk = zeros(chunk_size)
    apo = 0.0
    bef2 = 0.0
    bef1 = 0.0

    for c in 1:Int(Nt/chunk_size)
        chunk .= read(file, "/data/" * string(c))
        for i in 1:chunk_size
            if bef2 < bef1 > chunk[i]
                apo = chunk[i]
            elseif bef2 > bef1 < chunk[i]
                ec[ec_i] = (apo-chunk[i])/(apo+chunk[i])
                ec_i += 1
            end
            bef2 = bef1
            bef1 = chunk[i]
        end
    end
    
    return ec
end

calc_orbit_ec (generic function with 2 methods)

In [82]:
f = h5open("./results/dt1hr_tEnd500000yr/n_distance_sun_earth_bary.h5", "r")
# f1 and f2 must have the same params as each other's
N = read(f, "/params/n")
tEnd = read(f, "/params/tend")
dt = read(f, "/params/dt")
chunk_size = read(f, "/params/chunksize")

@time ec = calc_orbit_ec(N, f, tEnd, dt, chunk_size, 500000)

close(f) 

 15.147061 seconds (150.01 k allocations: 32.670 GiB, 9.07% gc time)


In [96]:
#delete zeros
_end = length(ec)
while ec[_end] == 0
    _end -= 1
end

p = plot(
    (@view ec[1:_end]),
    dpi=300,
    legend=false
)

#display(p)
savefig(p, "ecc_evol.png")

"/Users/shim/Desktop/Tanq/Planet/Gravitational_Simulation/ecc_evol.png"